# SPARK SCENARIOS 

In [44]:
from pyspark.sql import SparkSession,DataFrame
import pyspark.sql.functions as F

In [2]:
spark=SparkSession.builder.\
appName("spark_scenarios").\
getOrCreate()

In [3]:
# %config Completer.use_jedi = False
# !pip install --upgrade jedi==0.17.2

1. Recursively read file from folders

a. Using recursiveFileLookup option in spark 3.0 +

In [4]:
spark.read.options(recursiveFileLookup=True,inferSchema=True,delimiter="\t").\
csv("/user/itv452844/test_data/recursive_read/hr_db").count()

25

In [5]:
# ! hadoop fs -cat /user/itv452844/test_data/recursive_read/hr_db/part1/part-m-00000

2. Replace Values from an arrayType Column

In [27]:
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,ArrayType,IntegerType

* create test dataframe with arrayType column

In [47]:
schema=StructType([StructField('id',IntegerType()),StructField('val',ArrayType(StringType()))])

row={'id':1,'val':['null','2','null','2','4']}

df=spark.createDataFrame([Row(**row)],schema=schema)

df.toPandas() # input dataframe

,id,val
0,1,"[null, 2, null, 2, 4]"


In [49]:
df.createOrReplaceTempView("dfTable") # regestering as a temp view for querying

In [52]:
# query for replacing elements from a arrayType Column
qry="""
select 
id,
split(replace(concat_ws(',',val),'null',''),',') as val
from
dfTable
"""
spark.sql(qry)

id,val
1,"[, 2, , 2, 4]"
